# Imports

In [ ]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import plotly.express as px
import polars as pl
import polars.selectors as cs
from plotly.subplots import make_subplots

import patio.model.colo_results as cr
from patio.model.colo_common import FANCY_COLS, order_columns, pl_str_contains

In [ ]:
%reload_ext autoreload

# Results

## Setup

In [ ]:
write_figs = False
run = "202507170146"

In [ ]:
results = cr.Results(
    "202505261158",
    "202507031241",
    "202507070053",
    "202507130115",
    "202507170146",
    max_ppa=200,
    max_attr_rev_export_clean=100,
    max_violation=0.3,
    font="Roboto",
)
results.summary_stats()

In [ ]:
# results.update_params(max_violation=0.5, max_ppa=200)
results["202507170146"].filter(pl.col("good")).group_by("name", "regime").agg(
    pl.col("load_mw").sum()
)

## Tables

In [ ]:
results.for_xl("all", clip=False).pipe(order_columns).write_clipboard()

In [ ]:
results.for_dataroom(clip=True)

In [ ]:
sorted(results.all_summaries()["run"].unique())

## Supply curve

In [ ]:
sup_curve = results.fig_supply_curve("all", land_screen=False)
if write_figs:
    sup_curve.write_image(
        results.fig_path / f"{run}_supply_curve.pdf",
        height=580 * 0.8,
        width=1300 * 0.8,
    )
sup_curve.show()

## Opportunity map

In [ ]:
op_map = results.fig_scatter_geo("all", sixe_max=15, land_screen=False).update_layout(
    height=1100
)
if write_figs:
    op_map.write_image(results.fig_path / f"{run}_best_map.pdf", width=1600, height=1200)
op_map.show()

## Selection map

In [ ]:
sel_map_fig = results.fig_selection_map(run, land_screen=False)
if write_figs:
    sel_map_fig.write_image(results.fig_path / f"{run}_sel_map.pdf", width=1600, height=1200)
sel_map_fig.show()

## Case sheets

In [ ]:
if write_figs:
    results.case_sheets(run, land_screen=False)
else:
    results.fig_case_subplots(run, land_screen=False)

## Package as csvs

In [ ]:
results.compare(
    "colo_202505261158",
    "colo_202507170146",
    values=(
        "run_status",
        "load_mw",
        "served_pct",
        "pct_load_clean",
        "ppa_ex_fossil_export_profit",
        "attr_rev_export_clean",
        "max_rolling_violation_pct",
        "good",
    ),
).filter(
    pl.col("tech").is_in(("ngcc", "nggt"))
    & pl.col("state").is_in(("NM", "CO", "AZ", "UT", "ID"))
    & (pl.col("run_status") == "SUCCESS")
).sort(
    "balancing_authority_code_eia",
    "state",
    "icx_id",
    "name",
    "regime",
).write_clipboard()

## Exploring indicator distributions

In [ ]:
px.histogram(
    results.all_summaries().filter(
        (pl.col("new_fossil_mw") > 0) & (pl.col("run_status") == "SUCCESS")
    ),
    x="max_rolling_violation_pct",
    log_y=True,
    nbins=100,
    facet_col="name",
    facet_row="run",
)

In [ ]:
px.histogram(
    results.summaries[run].filter(
        (pl.col("new_fossil_mw") > 0) & (pl.col("run_status") == "SUCCESS")
    ),
    x="max_rolling_violation_pct",
    log_y=True,
    nbins=100,
    facet_col="name",
)

In [ ]:
pl.from_pandas(results.ad.gens).filter(pl.col("category") == "proposed_clean").with_columns(
    pl.col("operating_date").dt.year()
).pivot(
    on="technology_description",
    index="operating_date",
    values="capacity_mw",
    aggregate_function="sum",
).sort("operating_date")

In [ ]:
self = results
cols = self.id_cols
a = "202505261158"
b = "202507170146"
cols = ("ba_code", "icx_id", "icx_gen", "tech")
metric = ("attr_rev_export_clean", "ppa_ex_fossil_export_profit")

In [ ]:
# data = self.all_summaries().filter(pl.col("run").is_in((a, b))).pivot(on='run', index=self.id_cols, values=metric).filter(pl.col(a).is_not_null() | pl.col(b).is_not_null()).with_columns(pl.col(a, b).fill_null(0.0))
data = (
    self.all_summaries()
    .filter(pl.col("run").is_in((a, b)) & pl.col("best_at_site"))
    .unpivot(on=metric, index=("run", *cols))
    .pivot(on="run", index=(*cols, "variable"), values="value")
    .filter(pl.col(a).is_not_null() | pl.col(b).is_not_null())
    .with_columns(pl.col(a, b).fill_null(0.0))
)
# dmax = data.select(pl.max_horizontal(pl.max(a), pl.max(b)))
# dmin = data.select(pl.min_horizontal(pl.min(a), pl.min(b)))
fig = (
    (
        px.scatter(
            data,
            x=a,
            y=b,
            template="ggplot2",
            facet_col="variable",
        )
        # .update_xaxes(range=[dmin, dmax]).update_yaxes(range=[dmin, dmax])
        .update_xaxes(constrain="domain", matches=None)
        .update_yaxes(matches=None, showticklabels=True)
    )
    .for_each_yaxis(lambda x: x.update(scaleanchor=x.anchor))
    # .for_each_yaxis(lambda x: print(x))
    # .for_each_xaxis(lambda x: print(x))
    .for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
)
list(fig.layout)

## Logs

## Util screening

In [ ]:
aligned = results.get_aligned("202507170146")

In [ ]:
aligned.filter(
    pl_str_contains(
        "utility_name_eia_lse",
        "Ameren",
        "Evergy",
        "Georgia Power Co",
        "Alabama Power Co",
        "Mississippi Power Co",
        "Arizona Public Service",
        "Duke Energy Progress",
        "Duke Energy Carolinas, LLC",
        "Interstate Power and Light Co",
        "Wisconsin Power & Light Co",
    )
)["parent_name"]

In [ ]:
pl.read_parquet("/Users/alex/patio_data/colo_202507180034/colo_summary.parquet")["latitude"]